In [130]:
import pandas as pd
import string
from nltk.corpus import stopwords
from itertools import chain
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [143]:
df = pd.read_csv('sites.csv', encoding='latin')

In [144]:
df

,site,description
0,Gitlab.com,From project planning and source code manageme...
1,Cloudbees.com,"Reduce risk, optimize software delivery and ac..."
2,squarespace..com,Squarespace is the all-in-one solution for any...
3,Wix.com,Create a free website with Wix.com. Choose a s...


In [145]:
df.to_csv('sites1.csv', encoding='utf-8')

In [146]:
df = pd.read_csv('sites1.csv')

In [147]:
df

,Unnamed: 0,site,description
0,0,Gitlab.com,From project planning and source code manageme...
1,1,Cloudbees.com,"Reduce risk, optimize software delivery and ac..."
2,2,squarespace..com,Squarespace is the all-in-one solution for any...
3,3,Wix.com,Create a free website with Wix.com. Choose a s...


In [148]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,site,description
0,Gitlab.com,From project planning and source code manageme...
1,Cloudbees.com,"Reduce risk, optimize software delivery and ac..."
2,squarespace..com,Squarespace is the all-in-one solution for any...
3,Wix.com,Create a free website with Wix.com. Choose a s...


In [149]:
def cleanup(text):
    #removing punctuations
    no_punc = [char for char in text if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    #removing the stopwords
    clean_info = [word for word in no_punc.split() if word.lower() not in stopwords.words('english')]
    return clean_info

In [150]:
df['description'] = df['description'].apply(cleanup)

In [151]:
def listToString(s):  
    
    str1 = ""  
    
    for ele in s:  
        str1 += ele + " "   
    
    return str1  
df['description'] = df['description'].apply(listToString)

In [152]:
df

,site,description
0,Gitlab.com,project planning source code management CICD m...
1,Cloudbees.com,Reduce risk optimize software delivery acceler...
2,squarespace..com,Squarespace allinone solution anyone looking c...
3,Wix.com,Create free website Wixcom Choose stunning tem...


In [153]:
cv = TfidfVectorizer()

In [154]:
count_matrix = cv.fit_transform(df['description'])

In [155]:
cos_sim = cosine_similarity(count_matrix)

In [157]:
def get_index_from_site(name):
    return df.index[df.site==name].values
def get_site_from_index(m_index):
    return df[df.index==m_index]['site'].values

In [158]:
site_name='Wix.com' #site on which the recommendation will be calculated and displayed
site_index = get_index_from_site(site_name) #getting the index of the site from the site name
similar_sites = cos_sim[site_index] #fetching the similarity scores of all the sites descriptions against the given 'site_name' from the 'cosine_similarity' matrix
similar_sites = list(enumerate(chain.from_iterable(similar_sites))) #Converting the resulting list of similarity scores into a list of tuples where each tuple contains the individual movie's index on the 0th index and it's similarity score on the 1st index
sorted_sites = sorted(similar_sites, key= lambda x: x[1], reverse=True) 

In [164]:
i=0
for site in sorted_sites:
    print(get_site_from_index(site[0]))

['Wix.com']
['squarespace..com']
['Gitlab.com']
['Cloudbees.com']
